In [ ]:
# seed value
# to obtain reproducable results set random seed at the beginning of the script
# Apparently you may use different seed values at each stage
seed_value = 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as back
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
back.set_session(sess)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, TimeDistributed

In [ ]:
class MY_RNN():
    
    def __init__(self, hidden_units: int, n_features: int, n_timesteps, n_labels: int, loss: str, dropout = False, optimizer):
        self.hidden_units = hidden_units
        self.n_features = n_features
        self.n_timesteps = n_timesteps
        self.n_labels = n_labels
        self.loss = loss
        self.dropout = dropout
        self.optimizer = optimizer
        
        # 2 hidden layers, where the second hidden layer has twice as many hidden units than the first
        # 1 output layer with softmax as activation function (output = probability distribution over the labels)
        self.model = Sequential()
        self.model.add(SimpleRNN(hidden_units, return_sequences = True), input_shape = (n_timesteps, n_features))
        
        if self.dropout != False:
            self.model.add(Dropout(dropout))
        
        self.model.add(RNN(hidden_units * 2, return_sequences = False))
        
        if self.dropout != False:
            self.model.add(Dropout(dropout))
        
        
        self.model.add(Dense(n_labels, activation = 'softmax'))
        
        self.model.compile(loss = 'categorical_crossentropy', optimizer = self.optimizer, metrics = ['accuracy'])

In [ ]:
class MY_LSTM():
    
    def __init__(self, hidden_units: int, n_features: int, n_timesteps, n_labels: int, loss: str, dropout = False, optimizer):
        self.hidden_units = hidden_units
        self.n_features = n_features
        self.n_timesteps = n_timesteps
        self.n_labels = n_labels
        self.loss = loss
        self.dropout = dropout
        self.optimizer = optimizer
        
        # 2 hidden layers, where the second hidden layer has twice as many hidden units than the first
        # 1 output layer with softmax as activation function (output = probability distribution over the labels)
        self.model = Sequential()
        self.model.add(LSTM(hidden_units, return_sequences = True), input_shape = (n_timesteps, n_features))
        
        if self.dropout != False:
            self.model.add(Dropout(dropout))
        
        self.model.add(LSTM(hidden_units * 2, return_sequences = False))
        
        if self.dropout != False:
            self.model.add(Dropout(dropout))
        
        
        self.model.add(Dense(n_labels, activation = 'softmax'))
        
        self.model.compile(loss = 'categorical_crossentropy', optimizer = self.optimizer, metrics = ['accuracy'])

In [ ]:
def get_bidirectional(hidden_units: int, n_labels: int, n_features: int, n_timesteps = None, mode = 'concat'):
    
    """
    Return_sequences in the last hidden LSTM layer is required to be set to false, as we want to predict a label
    for each document. Output (softmax) layer computes a probability distribution over all k classes for each document.
    It predicts the most probable class for each document.
    You can change the mode argument (for the bidirectional computation) to any other possible option. 
    """
    
    model = Sequential()
    model.add(Bidirectional(LSTM(hidden_units, return_sequences = True), input_shape = (n_timesteps, n_features), merge_mode = mode))
    model.add(Dropout(0.1))
    model.add(LSTM(hidden_units * 2, return_sequences = False))
    model.add(Dropout(0.1))
    model.add(Dense(n_labels, activation = 'softmax'))
    adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0, amsgrad=False)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [ ]:
# important variables to reshape data for LSTM
n_samples = Xtrain.shape[0]
n_timesteps = Xtrain.shape[1]

try:
    n_features = Xtrain.shape[2]
except:
    n_features = dims

In [ ]:
n_units = 20
n_labels = K
n_batches = 64
n_epochs = 10